In [ ]:
# Modules, functions -- 
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import functions as fcns # custom functions Artem/Chris wrote


pd.set_option('max_columns', 500)
pd.set_option('max_rows', 1700)

In [ ]:
file_names = np.load('data/file_names.npy', allow_pickle=True).tolist()
articles_of_interest = np.load('data/articles_of_interest.npy', allow_pickle=True).tolist()


GAS_corrected_net_qty = file_names[0]
seasonality_based_forecasts = file_names[1]

preds = pd.read_excel(seasonality_based_forecasts) 
dat_GAS = pd.read_excel(GAS_corrected_net_qty) 


In [ ]:
# Combine weekly (1) seasonality predictions and (2) GAS corrections
preds = pd.merge(preds, dat_GAS.round(), how = 'left', on =['article_number', 'year', 'week'])

# Weekly forecast decision rules
preds['y_hat'] = np.where(np.isnan(preds.GAS_est), preds.seas_preds, (preds.GAS_est + preds.seas_preds)/2) # Evgeniy step
preds['y_hat'] = np.where((~np.isnan(preds.net_qty)) & (preds.y_hat < preds.net_qty), preds.net_qty, preds.y_hat) # Risk management step (judgment call)


In [ ]:
preds_season = pd.DataFrame(preds.groupby('article_number')['y_hat', 'net_qty'].apply(sum).round()) # Sum over season

In [ ]:
growth_rate = 1.10
preds_season['y_hat'] = preds_season.y_hat * growth_rate # default growth rate

In [ ]:
# # Combine DAA + eCom forecasts -- 
# ecom = pd.read_excel('data/ecom.xlsx')
# preds_season = pd.merge(preds_season, ecom, how = 'left', on = 'article_number').round()
# articles_of_interest = np.load('articles_of_interest.npy', allow_pickle=True).tolist()


In [ ]:
cost_margin = pd.read_csv('data/run_data.csv', usecols=['article_number', 'price', 'cost', 'margin']).drop_duplicates()

cost_margin = cost_margin[cost_margin.article_number.isin(articles_of_interest)]

cost_margin = (pd.DataFrame(cost_margin.groupby('article_number')['price', 'cost', 'margin'].mean().round(2)).dropna()) # All but one NA is season_net_qty < 10


In [ ]:
preds_season = pd.merge(preds_season, cost_margin, how = 'left', on = 'article_number')

In [ ]:

preds_season_w_price_cost_margin = file_names[2]

# w/ price, cost, margin for optimal overbuy
preds_season.to_excel(preds_season_w_price_cost_margin, index = True) 